In [1]:
import pandas as pd
import requests

In [2]:
sleeper_data = requests.get('https://api.sleeper.app/v1/players/nfl')

In [15]:
player_data = pd.DataFrame(sleeper_data.json()).T

In [ ]:
## Antes de quitar puntuacion 1162
## Despues de puntuacion 838
## Mas merges 800

In [35]:
import re

player_data['first_name'] = player_data['first_name'].str.replace(r'[^a-zA-Z]', '', regex=True)
player_data['last_name'] = player_data['last_name'].str.replace(r'[^a-zA-Z]', '', regex=True)

In [36]:
player_data['Name'] = player_data['first_name'] + player_data['last_name']

In [44]:
player_data = player_data.reset_index(names=['sleeper_id'])

In [18]:
combo2 = pd.read_csv("../data/2023_model_data_1_PPR.csv")

In [52]:
df = combo2.merge(player_data[['Name','sleeper_id']],on='Name',how='left')[['Name','sleeper_id','position']]

In [63]:
from fuzzywuzzy import process

def fuzzy_merge(df1, df2, key1, key2, threshold=90, limit=1):
    """
    df1 is the left table to join
    df2 is the right table to join
    key1 is the key column of df1
    key2 is the key column of df2
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df2[key2].tolist()
    
    m = df1[key1].apply(lambda x: process.extract(x, s, limit=limit))
    df1['matches'] = m
    
    m2 = df1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df1['matches'] = m2
    
    return df1

def merge_player_data(combo2, player_data):
    # First, perform the initial merge
    df = combo2.merge(player_data[['Name','sleeper_id']], on='Name', how='left')[['Name','sleeper_id','position']]
    list_fuzzy = []
    # Identify unmatched players
    unmatched = df[df['sleeper_id'].isna()]
    
    # Perform fuzzy merge for each position
    positions = unmatched['position'].unique()
    
    for pos in positions:
        pos_unmatched = unmatched[unmatched['position'] == pos]
        pos_player_data = player_data[player_data['position'] == pos]
        
        pos_fuzzy = fuzzy_merge(pos_unmatched, pos_player_data, 'Name', 'Name')
        pos_fuzzy = pos_fuzzy.merge(pos_player_data[['Name', 'sleeper_id']], left_on='matches', right_on='Name', how='left')
        pos_fuzzy = pos_fuzzy.drop(['matches', 'Name_y'], axis=1)
        pos_fuzzy = pos_fuzzy.rename(columns={'sleeper_id_y': 'sleeper_id', 'Name_x': 'Name'})
        list_fuzzy.append(pos_fuzzy)
    df_fuzzy = pd.concat(list_fuzzy,ignore_index=True)
    return df,df_fuzzy

In [64]:
df,df_fuzzy = merge_player_data(combo2, player_data)

C:\Users\isaac\AppData\Local\Temp\ipykernel_23040\1967388867.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['matches'] = m
C:\Users\isaac\AppData\Local\Temp\ipykernel_23040\1967388867.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['matches'] = m2
C:\Users\isaac\AppData\Local\Temp\ipykernel_23040\1967388867.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [85]:
df = combo2.merge(player_data[['Name','sleeper_id']], on='Name', how='left')[['Name','sleeper_id','position']]

In [89]:
for name in df['Name'].unique():
    try:
        id_fuzzy = df_fuzzy[df_fuzzy['Name']==name]['sleeper_id'].iloc[0]
        df.loc[df['Name']==name,'sleeper_id'] = id_fuzzy
    except:
        pass

In [88]:
df_fuzzy[df_fuzzy['Name']=='AmonRaSt']['sleeper_id']

1      7547
11     7547
339    7547
Name: sleeper_id, dtype: object

In [ ]:
df.loc[df['Name']==name,'sleeper_id'] = id_fuzzy

In [92]:
df[df['sleeper_id'].isna()]

,Name,sleeper_id,position
234,RandyMoss,NaN,WR
429,JosephAddai,NaN,RB
456,HinesWard,NaN,WR
491,RickyWilliams,NaN,RB
519,ThomasJones,NaN,RB
...,...,...,...
7721,KyleBoller,NaN,QB
7729,BrodieCroyle,NaN,QB
7747,ChrisSimms,NaN,QB
7749,CurtisPainter,NaN,QB


In [102]:
df_fuzzy_names = df_fuzzy[df_fuzzy['sleeper_id'].isna()==False].drop_duplicates()